# Protocolo BB84 — Overview (resumen)

Consigna: modelar y experimentar la distribución de claves cuánticas BB84, evaluando ataques y ruido, y completar la cadena clásica (corrección de errores + amplificación de privacidad) para obtener una clave segura compartida.

## Qué ofrece
- Simulación end‑to‑end de BB84 con Qiskit Aer (density_matrix) y semilla reproducible.
- Ataque de Eve (intercept‑resend) con probabilidad configurable.
- Ruido configurable: depolarizing, bit/phase flip, phase/amplitude damping, thermal relaxation, y error de lectura (p0→1/p1→0).
- Sliders interactivos para ajustar parámetros (N, semilla, P(Eve), m, tipo/intensidad de ruido, readout p0→1/p1→0, bloque/rondas/seguridad/target, barrido) y ver en vivo su impacto en QBER, clave cribada y P(det).
- Métricas clave: QBER, longitud de clave cribada, prob. de detección, causas de discrepancia (Eve/Ruido/Aleatorio).
- Reconciliación con Cascade (paridades, búsqueda binaria, fugas, errores residuales).
- Amplificación de privacidad con SHAKE‑256 y parámetro de seguridad/target opcional.
- Interfaz interactiva en 4 partes (básico → Eve → ruido → pipeline completo).

## Pipeline
1) Preparación/ataque: Alice prepara (Z/X), opcional Eve intercepta y reenvía, Bob mide (Z/X).
2) Sifting: se conservan bases coincidentes; se obtienen clave cribada y QBER; estimación P(det) con muestreo m.
3) Ruido: aplicación de NoiseModel (compuertas $\{I,X,H\}$) y/o error de lectura según parámetros.
4) Reconciliación (Cascade): bloques y paridades; corrección por búsqueda binaria; se contabiliza la fuga.
5) Privacidad (SHAKE‑256): se limita longitud utilizable y se deriva clave final segura.

## Métricas
- Clave cribada/final, QBER, P(det), fugas y errores residuales.

## Supuestos
- Un disparo por bit (shots=1), semilla opcional; modelos de ruido de Qiskit Aer (aprox. de hardware real).

In [1]:
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
        !pip install quantum-jam-chirimbolo/
    %cd quantum-jam-chirimbolo


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from src import setup_part1, setup_part2, setup_part3, setup_part4

plt.style.use("seaborn-v0_8-darkgrid")
pd.options.display.max_rows = 200

## Parte 1 — Ejecución básica de BB84

Ajusta el número de bits generados por Alice y observa cómo se producen las coincidencias de bases y la clave cribada compartida.


In [ ]:
setup_part1()

## Parte 2 — Ataque de Eve y muestreo de detección

Se incorpora una semilla configurable para reproducir experimentos, además de un ataque de interceptación-resend por parte de Eve. Ajusta la probabilidad de interceptación y la cantidad de bits `m` usados para detección.


In [ ]:
setup_part2()

## Parte 3 — Explorando el impacto del ruido en BB84

Esta sección extiende la anterior e introduce canales de ruido configurables mediante `AerSimulator`. Ajusta el tipo de ruido y observa cómo varían el QBER y la probabilidad de detección.


In [ ]:
setup_part3()

## Parte 4 — Corrección de errores y amplificación de privacidad

Aplicamos el algoritmo Cascade para corregir discrepancias residuales y posteriormente ejecutamos privacidad amplificada con SHAKE-256 truncado.


In [ ]:
setup_part4()

## Conclusiones y próximos pasos

- Explora valores más altos de `N` para obtener estadísticas más estables.
- Ajusta `m`, las rondas de Cascade y el parámetro de seguridad para equilibrar detección y longitud final.
- Conecta el cuaderno a un backend real de IBM Quantum para contrastar con hardware físico.
